In [1]:
%pip install earthengine-api -q
%pip install folium -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee

In [3]:
ee.Authenticate()


Successfully saved authorization token.


In [5]:
ee.Initialize()

Utility functions

In [7]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name, opacity = 0.5):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
        opacity=opacity,
    ).add_to(self)
    
folium.Map.add_ee_layer = add_ee_layer

In [274]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Keep the columns of interest.
    df = df[['longitude','latitude',  *list_of_bands]]

    return df

def get_ee_collection_data(collection: ee.Collection, position: tuple[float, float], scale: int, bands: list[str]) -> pd.DataFrame:
    point = ee.Geometry.Point(*position)
    data = collection.getRegion(point, scale).getInfo()
    return ee_array_to_df(data, bands)

def get_ee_image_data(image: ee.Image, position: tuple[float, float], scale: int, bands: list[str]) -> pd.DataFrame:
    collection = ee.ImageCollection([image])
    point = ee.Geometry.Point(*position)
    data = collection.getRegion(point, scale).getInfo()
    return ee_array_to_df(data, bands)

In [185]:
from IPython.display import display

def display_map(position, image: ee.Image, band_viz_palette=None, band_viz_min=0, band_viz_max=0.05, zoom=10):
    if not band_viz_palette:
        band_viz_palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
    band_viz = {
        'palette': band_viz_palette,
        'min': band_viz_min,
        'max': band_viz_max,
    }
    my_map = folium.Map(location=position, zoom_start=zoom)
    my_map.add_ee_layer(image, band_viz, 'S5P CO', opacity=0.4)
    display(my_map)

def display_dynamic_world(position, image, band_viz_min=0, band_viz_max=1, zoom=10):
    my_map = folium.Map(location=position, zoom_start=zoom)
    my_map.add_ee_layer(image, {"min": band_viz_min, "max": band_viz_max}, "Dynamic World", opacity=0.4)
    display(my_map)

Dynamic World

In [188]:
def get_built_dynamic_world(date_filter):
    CLASS_NAMES = [
        "water",
        "trees",
        "grass",
        "flooded_vegetation",
        "crops",
        "shrub_and_scrub",
        "built",
        "bare",
        "snow_and_ice",
    ]
    CLASS_PALETTE = [
        "000000",
        "000000",
        "000000",
        "000000",
        "000000",
        "000000",
        "red",
        "000000",
        "000000",
    ]
    dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    dwImage = ee.Image(dw.filter(date_filter).median())
    dwRgb = dwImage.select('label').visualize(min=0, max=8, palette=CLASS_PALETTE).divide(255)
    top1Prob = dwImage.select(CLASS_NAMES).reduce(ee.Reducer.max())
    top1ProbHillshade = ee.Terrain.hillshade(top1Prob.multiply(100)).divide(255)
    return dwRgb.multiply(top1ProbHillshade).select("vis-red")

In [298]:
date_filter = ee.Filter.date('2022-01-01', '2022-03-01')
colombia = (1.7, -74.8)
band = 'CO_column_number_density'

In [299]:
gas_collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').select(band).filter(date_filter)
gas_collection_median = gas_collection.median()

In [284]:
build_bool = get_built_dynamic_world(date_filter)
# build_bool = build_bool.multiply(ee.Image.constant(.5)).add(ee.Image.constant(1))

In [293]:
kernel = ee.Kernel.circle(10)
gas_collection_convoluted = gas_collection_median.convolve(kernel)
build_bool_convoluted = build_bool.convolve(kernel)

In [336]:
gas_collection_impact = gas_collection_median.subtract(gas_collection_convoluted)
gas_collection_impact = gas_collection_impact.multiply(ee.Image.constant(10**3))
# gas_collection_impact = gas_collection_impact.multiply(build_bool_convoluted)
# gas_collection_impact = gas_collection_impact.add(ee.Image.constant(.003))

In [290]:
colombia = (1.7, -74.8)
china = (23, 113)

In [ ]:
display_map(colombia, gas_collection_median)

In [ ]:
display_map(colombia, gas_collection_convoluted)

In [335]:
display_map(china, gas_collection_impact, band_viz_palette=["black", "blue", "purple", "red"]) #  band_viz_palette=["black", "blue", "purple", "red"]

In [ ]:
display_dynamic_world(china, build_bool_convoluted)
display_dynamic_world(china, build_bool)

In [222]:
gas_collection_convoluted.getInfo()["bands"]

[{'id': 'CO_column_number_density',
  'data_type': {'type': 'PixelType', 'precision': 'double'},
  'crs': 'EPSG:4326',
  'crs_transform': [1, 0, 0, 0, 1, 0]}]

In [333]:
build_bool_convoluted.getInfo()["bands"]

[{'id': 'vis-red',
  'data_type': {'type': 'PixelType', 'precision': 'double'},
  'crs': 'EPSG:4326',
  'crs_transform': [1, 0, 0, 0, 1, 0]}]

In [334]:
get_ee_image_data(build_bool_convoluted, china, 1, ["vis-red"])

KeyboardInterrupt: 

In [330]:
get_ee_image_data(gas_collection_convoluted, colombia, 1, ["CO_column_number_density"])

,longitude,latitude,CO_column_number_density
0,1.700003,-74.799996,0.008148


In [ ]:
get_ee_collection_data(gas_collection, colombia, 1, ["CO_column_number_density"])

In [295]:
get_ee_image_data(gas_collection_impact, colombia, 1, ["CO_column_number_density"])

,longitude,latitude,CO_column_number_density
0,1.700003,-74.799996,-0.000004


In [327]:
import csv

with open("../latlon_data/hm.csv", "r") as file:
    reader = csv.reader(file)
    next(reader)
    for line in reader:
        position = tuple(map(lambda i: float(i), line[6:8]))
        df = get_ee_image_data(gas_collection_impact, position, 1, [band])
        if not df.empty:
            print(df[0])

0  longitude   latitude  CO_column_number_density
0  38.381513  27.191065             -1.908196e-16
0  longitude   latitude  CO_column_number_density
0  38.481999  27.047101              1.179612e-16
0  longitude   latitude  CO_column_number_density
0  37.519571  40.930635              1.179612e-16
0  longitude   latitude  CO_column_number_density
0  40.202416  29.188873              5.204170e-17
0 longitude   latitude  CO_column_number_density
0  41.04853  28.812389              1.595946e-16
0  longitude   latitude  CO_column_number_density
0  41.020997  28.812901              3.469447e-18
0  longitude   latitude  CO_column_number_density
0  39.712996  43.026118              1.040834e-16
0  longitude   latitude  CO_column_number_density
0  41.573093  27.766373             -1.734723e-16
0  longitude  latitude  CO_column_number_density
0  41.409491 -8.776114             -1.630640e-16
0  longitude latitude  CO_column_number_density
0  45.536235  9.63343              1.769418e-16
0  longi

KeyboardInterrupt: 